# CS124 and a bit, session 1: Graphs, DFS, SCCs, BFS, SSSP

This is a workbook that you can (but don't have to) use for the problems! They arae in the same order as the PDF, and generally have some basic test(s) set up for you.
___

### Problem 1: Switching graph representations

Here you need to implement two functions to convert between the adjacency matrix and adjacency list representations of a graph. You may assume for each that the input is a valid adjacency list/matrix (e.g. you do not need to check the matrix is square).

In [9]:
def list_to_matrix(adj_list: list[list[int]]):
    n = len(adj_list)
    adj_matrix = [[0] * n for _ in adj_list]
    for u in range(n):
        for v in adj_list[u]:
            adj_matrix[u][v] = 1
    return adj_matrix

def matrix_to_list(adj_matrix: list[list[int]]):
    adj_list = [[] for _ in adj_matrix]
    n = len(adj_matrix)
    for u in range(n):
        for v in range(n):
            if adj_matrix[u][v]:
                adj_list[u].append(v)
    return adj_list


In [10]:
# matrix_to_list testing - you can (and probably should) add your own tests!
matrix_1 = [[0, 1, 1, 0],
            [1, 0, 0, 0],
            [0, 0, 0 ,0],
            [1, 1, 0, 0]]
list_1 =  [[1, 2], [0], [], [0, 1]]

# TESTS -> UNCOMMENT BELOW TO TEST FUNCTIONS. ADD YOUR OWN TESTS :)

# assert(list_to_matrix(list_1) == matrix_1)
# assert(matrix_to_list(matrix_1) == list_1)

___

### Problem 2: DFS backedges

For this problem you need to write, and then use or modify DFS to print all backedges in a graph. To start with, you may assume that all vertices are reachable from vertex 0 if you'd like. There are multiple ways of implementing this - it is up to you how you choose to do so. It may be useful to you to write in the tex file I provided how you chose to do it, and any other methods you thought of that you could use to implement this and their tradeoffs, but that is optional (as is everything in this entire project in fact, but this one is even more optional than the others)

In [9]:
def find_back_edges(verts: list[int], edges: list[list[int]]):
    explored = set()
    current_path = set()
    def DFS(u): 
        explored.add(u)
        for v in edges[u]:
            if v in current_path:
                print(u, v)
            if not(v in explored):
                current_path.add(v)
                DFS(v)
        current_path.remove(u)
    for v in verts:
        if not(v in explored):
            current_path.add(v)
            DFS(v)

In [16]:
# Since back edges depend on DFS implementations, no precise testing here, just printing your results,
# you should check that these make sense.
verts = [0, 1, 2, 3, 4]
edges = [[1, 2], [3], [3, 1], [4, 1], [2]]
print("First test")
find_back_edges(verts, edges)

''' My implementation prints:
2 3
2 1
3 1
'''
print("\nSecond test")
find_back_edges(verts[::-1], edges)

''' My implementation prints:
3 4
1 3
'''


First test
2 3
2 1
3 1

Second test
3 4
1 3


' My implementation prints:\n3 4\n1 3\n'

___

### Problem 3: Number of islands

For this problem you are given a m x n integer grid, where each position is either a 0 or a 1. 1 represents a piece of land, while 0 represents water. Your job is to write a function to determine the number of islands in the grid, where an island is formed of adjacent pieces of land (adjacent horizontally or vertically, not diagonally)

In [4]:
def count_islands(grid: list[list[int]]):
    height = len(grid)
    width = len(grid[0])
    
    def DFS(y, x):
        if grid[y][x] == 0:
            return
        grid[y][x] = 0
        DFS(max(y - 1, 0), x)
        DFS(min(y + 1, height - 1), x)
        DFS(y, max(x - 1, 0))
        DFS(y, min(x + 1, width - 1))

    islands = 0
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 1:
                DFS(i, j)
                islands += 1
    return islands

In [8]:
import numpy as np
# Andrew's provided tests. Not exhausitve.
grid_1 = [[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0], 
        [0, 0, 1, 0, 0], 
        [0, 0, 0, 1, 1]]
assert(count_islands(grid_1) == 3)

grid_2 = [[abs(round(np.sin(x + y))) for x in range(6)] for y in range(6)]
assert(count_islands(grid_2) == 4)

___

### Problem 4: Lowest Common Ancestor of a Binary Tree

In this problem you need to find the lowest common ancestor of two values in a binary tree. You may assume that the two values are in the tree. To be a common ancestor of the two values, there must exist a path from the vertex to both vertices OR the vertex must be one of the target vertices, with a path to the other vertex. To be the lowest common ancestor, it must be a common ancestor of the two vertices that minimizes the sum of the path distances from the vertex to the two vertices.

In [1]:
class Node:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

def lowest_common_ancestor(root: Node, a: int, b: int):
    if not(root):
        raise ValueError
    LCA = root.val
    def DFS(node):
        nonlocal LCA
        current = node.val == a or node.val == b
        if not(node.left or node.right):
            return current
        left = DFS(node.left)
        right = DFS(node.right)
        if left + current + right == 2:
            LCA = node.val
        return left or current or right
    DFS(root)
    return LCA


In [2]:
# Andrew's provided LCA tests. Not exhaustive.
nodes = []
for i in range(0, 9):
    nodes.append(Node(i))
nodes[1].left = nodes[0]
nodes[1].right = nodes[8]
nodes[3].right = nodes[1]
nodes[3].left = nodes[5]
nodes[5].left = nodes[6]
nodes[5].right = nodes[2]
nodes[2].left = nodes[7]
nodes[2].right = nodes[4]

assert(lowest_common_ancestor(nodes[3], 5, 4) == 5)
assert(lowest_common_ancestor(nodes[3], 5, 1) == 3)


___

### Problem 5: Finding SCCs

For this problem you need to implement an SCC-finding algorithm (namely, Kosaraju's from class) to find the SCCs in a graph.

In [17]:
def find_SCCS(verts: list[int], edges: list[list[int]]):
    edges_reversed = [[] for v in verts]
    for u in verts:
        for v in edges[u]:
            edges_reversed[v].append(u)
    
    def DFS(verts, edges, print_SCC=False):
        explored = set()
        postorder_list = []
        def search(u):
            explored.add(u)
            for v in edges[u]:
                if not(v in explored):
                    search(v)
            postorder_list.append(u)
        for v in verts:
            if not(v in explored):
                search(v)
                if print_SCC:
                    print(f'New SCC: {postorder_list}')
                    postorder_list = []
        return postorder_list

    postorder_list = DFS(verts, edges_reversed)
    print(postorder_list)
    DFS(postorder_list[::-1], edges, print_SCC=True)

In [22]:
edges = [[1, 2], [3], [3, 1], [4, 1], [2]]
verts = [0, 1, 2, 3, 4]
find_SCCS(verts, edges)
find_SCCS(verts[::-1], edges)

[0, 3, 4, 2, 1]
New SCC: [2, 4, 3, 1]
New SCC: [0]
[0, 2, 1, 3, 4]
New SCC: [1, 3, 2, 4]
New SCC: [0]


___

### Problem 6: BFS

For this problem you must implement BFS to find the depth (or path distance, assuming all edges have length 1) to all vertices reachable from the source vertex. You should implement a queue yourself.

In [1]:
def BFS(verts: list[int], edges: list[list[int]], source=0):
    explored = set([source])
    q = [source]
    dist = [0 for _ in verts]
    while q:
        u = q.pop(0)
        for v in edges[u]:
            if not(v in explored):
                q.append(v)
                dist[v] = dist[u] + 1
                explored.add(v)
    return dist

In [2]:
edges = [[1, 2], [3], [3, 1], [4, 1], [2]]
verts = [0, 1, 2, 3, 4]
print(BFS(verts, edges))

[0, 1, 1, 2, 3]


### Problem 7: Dijkstra's


In [ ]:
# To come